In [3]:
import requests
head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
           'Chrome/59.0.3071.115 Safari/537.36'}

In [4]:
rss_reddit_clarin = 'https://www.reddit.com/domain/clarin.com/.rss?page=1&pagesize=100'
feed = requests.get(rss_reddit_clarin, headers = head)
content = feed.content
content

b'<?xml version="1.0" encoding="UTF-8"?><feed xmlns="http://www.w3.org/2005/Atom"><category term="clarin.com" label="r/clarin.com"/><updated>2018-10-30T13:07:15+00:00</updated><id>/domain/clarin.com/.rss?page=1&amp;pagesize=100</id><link rel="self" href="https://www.reddit.com/domain/clarin.com/.rss?page=1&amp;pagesize=100" type="application/atom+xml" /><link rel="alternate" href="https://www.reddit.com/domain/clarin.com/?page=1&amp;pagesize=100" type="text/html" /><title>clarin.com on reddit.com</title><entry><author><name>/u/edesur</name><uri>https://www.reddit.com/user/edesur</uri></author><category term="argentina" label="r/argentina"/><content type="html">&lt;table&gt; &lt;tr&gt;&lt;td&gt; &lt;a href=&quot;https://www.reddit.com/r/argentina/comments/9sik82/la_f\xc3\xa1brica_militar_de_aviones_entrega_tres_pampa/&quot;&gt; &lt;img src=&quot;https://b.thumbs.redditmedia.com/5n9cNFvIdTFWEncX3wxmKXojRsOy9zkHuPX3a-rIdFM.jpg&quot; alt=&quot;La f\xc3\xa1brica militar de aviones entrega t

In [5]:
feed.status_code

200

In [6]:
reddit_json = 'https://www.reddit.com/domain/clarin.com/new.json'

reddit_json_request = requests.get(reddit_json, headers = head)
reddit_json_response = reddit_json_request.content
reddit_json_response

b'{"kind": "Listing", "data": {"modhash": "", "dist": 25, "children": [{"kind": "t3", "data": {"approved_at_utc": null, "subreddit": "argentina", "selftext": "", "author_fullname": "t2_44or2", "saved": false, "mod_reason_title": null, "gilded": 0, "clicked": false, "title": "Carlos Melconian provocador: \\"\\u00bfHay que seguir minti\\u00e9ndole a la gente para ganar una elecci\\u00f3n?\\"", "link_flair_richtext": [], "subreddit_name_prefixed": "r/argentina", "hidden": false, "pwls": 6, "link_flair_css_class": "politica", "downs": 0, "thumbnail_height": 73, "hide_score": false, "name": "t3_9sjbo1", "quarantine": false, "link_flair_text_color": "dark", "author_flair_background_color": null, "subreddit_type": "public", "ups": 11, "domain": "clarin.com", "media_embed": {}, "thumbnail_width": 140, "author_flair_template_id": null, "is_original_content": false, "user_reports": [], "secure_media": null, "is_reddit_media_domain": false, "is_meta": false, "category": null, "secure_media_embed"

In [7]:
reddit_json_request.status_code

200

In [9]:
import requests
import json
from time import sleep

def gen_reddit_json_link(link_id):
    return 'https://www.reddit.com/domain/clarin.com/new.json?after=t3_' + link_id

def save_reddit_json(post_id, parsed_json):
    with open('./jsons/' + post_id + '.json', 'w') as f:
        json.dump(parsed_json, f)

def get_next_post_id(parsed_json):
    children = parsed_json['data']['children']
    next_post_id = children[len(children) - 1]['data']['id']
    return next_post_id

#cambiar el numero de postid en base al que esta en el json
def scrap_reddit(post_id = '9ra8xh'):
    print('scrapping:', post_id)
    print('waiting 2 min before requesting data...')
    sleep(3 * 60)
    print('wait done!')
    
    reddit_json_link = gen_reddit_json_link(post_id)
    
    print('requesting:', reddit_json_link)
    reddit_json_request = requests.get(reddit_json_link,  headers = head)
    
    if reddit_json_request.status_code == 200:
        print('request success!')
        print('parsing content')
        reddit_json_response = reddit_json_request.content
        parsed_json = json.loads(reddit_json_response)
        
        print('saving:', post_id)
        save_reddit_json(post_id, parsed_json)
        
        next_post_id = get_next_post_id(parsed_json)
        scrap_reddit(next_post_id)
    else:
        print('request failed with status:', reddit_json_request.status_code)
        print('sleeping for 3 minutes...')
        sleep(3 * 60)
        scrap_reddit(post_id)

if __name__ == "__main__":
    scrap_reddit()

scrapping: 9ra8xh
waiting 2 min before requesting data...
wait done!
requesting: https://www.reddit.com/domain/clarin.com/new.json?after=t3_9ra8xh
request success!
parsing content
saving: 9ra8xh
scrapping: 9pmtmo
waiting 2 min before requesting data...
wait done!
requesting: https://www.reddit.com/domain/clarin.com/new.json?after=t3_9pmtmo
request success!
parsing content
saving: 9pmtmo
scrapping: 9o4m7p
waiting 2 min before requesting data...
wait done!
requesting: https://www.reddit.com/domain/clarin.com/new.json?after=t3_9o4m7p
request success!
parsing content
saving: 9o4m7p
scrapping: 9ms5c3
waiting 2 min before requesting data...
wait done!
requesting: https://www.reddit.com/domain/clarin.com/new.json?after=t3_9ms5c3
request success!
parsing content
saving: 9ms5c3
scrapping: 9lc230
waiting 2 min before requesting data...
wait done!
requesting: https://www.reddit.com/domain/clarin.com/new.json?after=t3_9lc230
request success!
parsing content
saving: 9lc230
scrapping: 9jw3pz
waiting

IndexError: list index out of range

In [12]:
reddit_links = merge_scrapped_links('./reddit/')

In [13]:
from os import listdir
import json

def merge_scrapped_links(links_dir):
    all_links = []
    for links_file in listdir(links_dir):
        #print(links_file)
        with open(links_dir + links_file, 'r' , encoding='latin-1') as f:
            links = json.load(f)['data']['children']
            all_links = all_links + links
    return all_links

In [14]:
# id, created, title, url, downs, ups, score
from datetime import datetime

reddit_links_data = [entry['data'] for entry in reddit_links]
reddit_links_data = [
   {
       'id': data['id'],
       'title': data['title'],
       'url': data['url'],
       'created': datetime.fromtimestamp(data['created']),
       'score': data['score'],
       'downs': data['downs'],
       'ups': data['ups']
   } for data in reddit_links_data
]

In [15]:
import pandas as pd
df_reddit_clarin = pd.DataFrame(reddit_links_data)
df_reddit_clarin.drop_duplicates(subset='url', inplace=True)
df_reddit_clarin.drop_duplicates(subset='title', inplace=True)

In [16]:
df_reddit_clarin.to_csv('./clarin_links.csv', index=False)

In [17]:
df = pd.read_csv('./clarin_links.csv')
df.sample(5)

,created,downs,id,score,title,ups,url
211,2018-06-02 04:57:19,0,8nwnim,42,Confirman que Alberto Nisman fue asesinado por...,42,https://www.clarin.com/politica/confirman-albe...
56,2018-04-11 21:57:44,0,8binbo,28,"Cayó el límite a los vuelos en el aeropuerto ""...",28,https://www.clarin.com/sociedad/cayo-limite-vu...
48,2018-04-07 03:48:39,0,8adtm5,15,Desde La Cámpora hasta Pablo Moyano marcharon ...,15,https://www.clarin.com/politica/campora-pablo-...
341,2018-07-11 04:52:49,0,8xum2c,10,El amor por sus abuelos y un recuerdo trágico:...,10,https://www.clarin.com/policiales/amor-abuelos...
92,2018-04-25 21:26:21,0,8euw2f,14,Una auditoría descubrió un contrato millonario...,14,https://www.clarin.com/politica/auditoria-desc...


In [18]:
import requests
import json
from time import sleep
import pandas as pd

save_folder = './htmls/'

def process_line(s):
    print('waiting 10 seconds before requesting data...')
    sleep(10)
    print('wait done!')
    
    print('requesting:',  s['url'])
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    clarin_request = requests.get(s['url'], headers)
    
    if clarin_request.status_code == 200:
        print('request success!')
        print('saving file as:', save_folder + s['id'] + '.html')

        with open(save_folder + s['id'] + '.html', 'w', encoding='utf-8') as file:
            file.write(clarin_request.text)
    else:
        print('request failed with status:', clarin_request.status_code)

    s['status'] = clarin_request.status_code
    return s

if __name__ == "__main__":
    df = pd.read_csv('./clarin_links.csv')
    df = df.apply(process_line, axis=1)
    df.to_csv('./clarin_links_processed.csv', index=False)

waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/vendedor-confiscaron-sandwiches-salame-revancha-acto-24-marzo_0_S1VjONNcG.html
request success!
saving file as: ./clarin/86wpto.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/vendedor-confiscaron-sandwiches-salame-revancha-acto-24-marzo_0_S1VjONNcG.html
request success!
saving file as: ./clarin/86wpto.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/deportes/automovilismo/argentina-volvera-tener-carrera-formula-2020_0_rkdvDJx5z.html
request success!
saving file as: ./clarin/86uf48.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/declaran-inconstitucional-decreto-macri-endurecio-controles-migratorios_0_BJIL1l79M.html
request success!
saving file as: ./clarin/86tpq3.html
waiting 10 seconds before requesting data...
wait done!
reque

wait done!
requesting: https://www.clarin.com/espectaculos/fama/jorge-rial-publico-twitter-telefono-natacha-jaitt-hizo-enfurecer_0_HyrXOuGjG.html
request success!
saving file as: ./clarin/89sjzm.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/espectaculos/fama/escandalo-natacha-jaitt-huyo-tribunales-preguntaron-dichos-mirtha-legrand_0_Syst-vMjG.html
request success!
saving file as: ./clarin/89qur3.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/echar-pedofilos-dijo-ex-psicologa-independiente_0_BkGpbIbjz.html
request success!
saving file as: ./clarin/89jjfq.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/polemica-pasajes-ahora-congreso-presionan-salariazo_0_rJbgGresz.html
request success!
saving file as: ./clarin/89d7qk.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/suiz

wait done!
requesting: https://www.clarin.com/economia/canasta-jubilados-subio-10-meses-supera-19000_0_Hyd49Qfhz.amp.html?__twitter_impression=true&amp;
request success!
saving file as: ./clarin/8cvxt4.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/policiales/llavallol-moreno-robos-terminaron-delincuentes-muertos_0_ByC129G3z.amp.html
request success!
saving file as: ./clarin/8cuvgu.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/instituto-balseiro-reconocio-dado-desaparecido-cientifico-vivo-ee-uu_0_HJwgr9z3M.html
request success!
saving file as: ./clarin/8csoyr.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/investigan-80-vuelos-exterior-milagro-sala-kirchnerismo_0_B1fVMU-3G.amp.html
request success!
saving file as: ./clarin/8cmt2q.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com

request success!
saving file as: ./clarin/8fcjms.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/policiales/detuvieron-menor-crimen-canillita-caseros_0_HJxwnelaG.html
request success!
saving file as: ./clarin/8fb8v3.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/argentina-primera-mujer-cupula-organizacion-meteorologica-mundial_0_BJoEwh33z.html
request success!
saving file as: ./clarin/8fsas9.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/opinion/entrevista-rector-uba-tomaramos-examen-ingreso-hoy-entraria-25_0_rkOK_LQaz.html
request success!
saving file as: ./clarin/8fs9qx.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/mundos-intimos-monte-santiagueno-hablaba-solo-quichua-anos-proteste-tener-usar-zapatos-aprender-espanol_0_HJv6EM-TM.html
request success!
saving file as: ./clari

request success!
saving file as: ./clarin/8iwi6f.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/viva/malvinas-hoy-sacan-minas-guerra_0_rkQ8sZMRz.html
request success!
saving file as: ./clarin/8iwfdl.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/eeuu-restaurantes-bares-puestos-comida-obligados-indicar-calorias-alimentos_0_Ska50GkRM.html
request success!
saving file as: ./clarin/8io4at.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/declaran-inocente-maestra-particular-hijo-alumno-14-anos_0_HJpNkHzRM.html
request success!
saving file as: ./clarin/8inqf9.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/policiales/policias-intentaron-asaltar-superior-comisario-ordeno-ocultar_0_HkP6umf0M.html
request success!
saving file as: ./clarin/8ikvcd.html
waiting 10 seconds before requesting 

...

request success!
saving file as: ./clarin/9h6335.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/economia/presupuesto-2019-eximir-ganancias-jueces-costara-854-millones_0_pl0QOeOUf.html
request success!
saving file as: ./clarin/9h0hk9.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/eugenio-zaffaroni-causas-judiciales-cristina-kirchner-malvinas-macri_0_U50s92xV7.html
request success!
saving file as: ./clarin/9h03cv.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/bloquean-millones-euros-nueva-cuenta-secreta-lazaro-baez-exterior_0_op9E7Wa3b.html
request success!
saving file as: ./clarin/9h02gu.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/mundo/encuentran-camion-abandonado-157-cadaveres-cabian-morgue_0_SJNzgQpO7.html
request success!
saving file as: ./clarin/9grinm.html
waiting 10 

request success!
saving file as: ./clarin/9ixh2c.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/economia/pese-enfriamiento-economia-actividad-financiera-sigue-alza_0_jl-tPQZz1.html
request success!
saving file as: ./clarin/9jhywg.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/repudian-profesor-escuelas-sopa-letras-mensajes-politicos-macri-gato-vidal-miente-crucigrama_0_zvTM_Zyuc.html
request success!
saving file as: ./clarin/9jhybl.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/datos-oficiales-pobreza-alcanzo-27_0_qJOnSwUzB.html
request success!
saving file as: ./clarin/9jg7gh.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/viajes/flybondi-comenzo-venta-pasajes-punta_0_fPE3xR8wJ.html
request success!
saving file as: ./clarin/9jeq2t.html
waiting 10 seconds before requesting dat

wait done!
requesting: https://www.clarin.com/sociedad/cohete-elon-musk-lanzan-satelite-argentino-anticipara-desastres-naturales_0_OiU7Y13KM.html
request success!
saving file as: ./clarin/9kxghl.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/mundo/beto-ex-rockero-puede-demoler-donald-trump-texas_0_0Xy2-JgTl.html
request success!
saving file as: ./clarin/9kx7zm.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/docente-denuncia-echaron-colegio-religioso-decir-todes-clase_0_ga0HtM7Rd.html
request success!
saving file as: ./clarin/9lijra.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/ciudades/albergues-transitorios-legislatura-aprueba-permitir-trios-parejas-swinger_0_DeFaMEZhx.html
request success!
saving file as: ./clarin/9ldd5n.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/deportes/polide

request success!
saving file as: ./clarin/9n1liw.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/gobierno-frena-retroactivo-gas-hara-cargo-plan-pago-2021_0_16iv1avTY.html
request success!
saving file as: ./clarin/9n19lo.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/ciudades/video-vagon-subte-salio-calle-choco-camionetas_0_Ea7mMjBac.html
request success!
saving file as: ./clarin/9nkxke.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/campana-redes-heroes-julio-vido-milagro-sala-amado-boudou_0_77M2qsbe1.amp.html
request success!
saving file as: ./clarin/9nkakc.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/policiales/video-muestra-policia-robando-comida-super_0_mZofOm4TO.html
request success!
saving file as: ./clarin/9nix75.html
waiting 10 seconds before requesting data...
wait d

request success!
saving file as: ./clarin/9pb06x.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/sociedad/mensaje-viral-hombre-feminista-noche-cualquier-piba-posible-violador_0_pJjjQTKZB.html
request success!
saving file as: ./clarin/9p4njf.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/ciudades/caza-uber-limites-quemaron-supuesto-auto-app-termino-ligando-delante_0_lnemV4X4L.html
request success!
saving file as: ./clarin/9oz4ah.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/ciudades/buenos-aires-encarece-precio-metro-cuadrado-segundo-alto-region_0_UmxOH7sgW.html
request success!
saving file as: ./clarin/9pu931.html
waiting 10 seconds before requesting data...
wait done!
requesting: https://www.clarin.com/politica/caso-cuadernos-empresarios-buscan-regular-aportes-campana-comprometen-coimear-cartelizarse_0_6bGb0OVq3.html
request success!
saving

In [19]:
from bs4 import BeautifulSoup

def parse_clarin_html(html):
    article = {
        'section': '',
        'volanta': '',
        'headline': '',
        'bajada': '',
        'articleBody': ''
    }
    
    soup = BeautifulSoup(html, 'html.parser')
    
    # section
    section = soup.find('div', {'class': 'hd'})
    if section:
        article['section'] = section.text.strip()
    
    # volanta
    volanta = soup.find('p', {'class': 'volanta'})
    if volanta:
        article['volanta'] = volanta.text.strip()
        
    # headline
    headline = soup.find('h1', {'id': 'title'})
    if headline:
        article['headline'] = headline.text.strip()
    
    # bajada
    bajada = soup.find('div', {'class': 'bajada'})
    if bajada:
        article['bajada'] = bajada.text.strip()
    
    # articleBody
    articleBody = soup.find('div', {'class': 'body-nota'})
    if articleBody:
        article['articleBody'] = articleBody.text.strip()
    
    return article

In [20]:
from os import listdir
import pandas as pd

articles = []
base_folder = './htmls/'
for filename in listdir(base_folder):  
    #print(filename)
    with open(base_folder + filename, 'r', encoding='utf-8') as f:
        html = f.read()
        article = parse_clarin_html(html)
        articles.append(article)
        
df = pd.DataFrame(articles)      
df.to_csv('clarin_articles.csv', index=False)

In [21]:
art = pd.read_csv('clarin_articles.csv')
art

,articleBody,bajada,headline,section,volanta
0,"Lucas Rodríguez, integrante de la Selección Na...",Lucas Rodríguez hizo un video solicitándole al...,Sancionan a un jugador de Los Murciélagos por ...,de Deportes,Fútbol para ciegos
1,Luego de la histórica conquista en Indian Well...,El número 6 del mundo se acercó a una chica qu...,El gran gesto de Del Potro con una admiradora ...,de Deportes,Tenis
2,La Cámara de Diputados debatió ayer un proyect...,"Hasta ahora sólo es contravención, en la Ciuda...",Debaten en Diputados un proyecto para penaliza...,de Sociedad,Violencia de género
3,"¡Y algún día serán como Marcelo T. de Alvear, ...",Las autoridades del Colegio creen que pueden l...,El Nacional Buenos Aires espera arrancar su in...,de Sociedad,Ansiedad entre los aspirantes
4,"En su primera sesión ordinaria del año, la Cám...","Es Romina del Plá, del Partido Obrero. Habló c...",Una diputada de izquierda fue la única que no ...,de Política,Se aprobaron penas
5,"Perteneció a una “raza”de futbolistas que, al ...",Fue un símbolo del Huracán campeón del 73. Bri...,"Murió el ex futbolista René Houseman, campeón ...",de Deportes,Tenía 64 años
6,"En un duro revés para el Gobierno, la Justicia...",La Justicia consideró que no se justificaba el...,Declaran inconstitucional el decreto de Macri ...,de Política,Revés para el Gobierno
7,Será difícil escucharlo de la boca de algún fu...,"Es “un secreto a voces”, aunque desde el Gobie...",Argentina volverá a tener una carrera de Fórmu...,de Deportes,Automovilismo
8,El vendedor al que le decomisaron la mercaderí...,"Días atrás, una foto que lo mostraba triste mi...",El vendedor al que le confiscaron los sándwich...,de Sociedad,Quiere que le devuelvan la canasta
9,Con algo más de 10 años como la red social líd...,Muchos usuarios de todo el mundo comenzaron a ...,¿Facebook puede desaparecer? La red social fre...,de Tecnología,Un reinado en jaque según los expertos


In [22]:
art[art['section'] == 'de Política']

,articleBody,bajada,headline,section,volanta
4,"En su primera sesión ordinaria del año, la Cám...","Es Romina del Plá, del Partido Obrero. Habló c...",Una diputada de izquierda fue la única que no ...,de Política,Se aprobaron penas
6,"En un duro revés para el Gobierno, la Justicia...",La Justicia consideró que no se justificaba el...,Declaran inconstitucional el decreto de Macri ...,de Política,Revés para el Gobierno
16,Después de la masiva respuesta que obtuvieron ...,Es luego de las manifestaciones del último dom...,Los anti abortistas suben la apuesta y no desc...,de Política,Debate público
23,El lonko Facundo Jones Huala lleva nueve meses...,El líder mapuche lleva 9 meses detenido. Se al...,"Facundo Jones Huala en prisión: aislado, escuc...",de Política,Conflicto en la Patagonia
27,El estudio científico hecho sobre las muestras...,Las pruebas dieron positivas en 2 manifestante...,Caso Rafael Nahuel: las pericias determinaron ...,de Política,Conflicto en el Sur
28,Suele pasar. La dirigencia política muchas vec...,"Tras las críticas de Macri, se prepara una res...",Canje de pasajes: los diputados porteños y bon...,de Política,Congreso.
30,La diputada oficialista Elisa Carrió se refiri...,La diputada oficialista dijo: “Para mí son viá...,Carrió y la crítica de Macri por el canje de p...,de Política,Hacía bastante que no hablaba
33,En medio de la polémica por los canjes de pasa...,"Miguel Pichetto, jefe de los senadores peronis...","Tras la polémica por los pasajes, ahora en el ...",de Política,Sobresueldos
42,"Militantes kirchneristas y de izquierda, organ...","Militantes kirchneristas y de izquierda, organ...",Desde La Cámpora hasta Pablo Moyano marcharon ...,de Política,En el barrio porteño de Retiro
47,Horas antes de la llegada al país del jefe del...,"En una entrevista con un diario español, el Pr...","Antes de la visita de Mariano Rajoy, Mauricio ...",de Política,Relación bilateral
